# P2. Les vues SQL

## Définition d'une vue SQL  

Une vue n'est rien de plus qu'une instruction SQLite qui est stockée dans la base de données avec un nom associé. Il s'agit en fait d'une composition d'une table sous la forme d'une requête SQLite prédéfinie.

Une vue peut contenir toutes les lignes d'une table ou des lignes sélectionnées d'une ou plusieurs tables. Une vue peut être créée à partir d'une ou plusieurs tables qui dépendent de la requête SQLite écrite pour créer une vue.

## Code utilitaire

In [4]:
import os
import sqlite3
import pandas as pd   

In [5]:
from pathlib import Path

In [6]:
# Ouvre connection vers SQLite db
chemin_bdd = Path('db') / 'chinook.db'
conn = sqlite3.connect(chemin_bdd)

In [7]:
# Fonction permettant d'exécuter un requête SQL sur une BDD définie par sa connexion conn
def executer_requete(requete_sql, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(requete_sql)
        conn.commit()
    except sqlite3.Error as e:
        print("Erreur lors de l'execution de la requête")
        print(e)
        return
    cursor.close()


## Example de vue SQL: v_albums

La requête suivante permet d'obtenir une liste (simplifiée) des albums / artistes correspondants:  

In [8]:
# Requete sur les tables albums & artists
requete = """
    SELECT AlbumId,
           Title,
           a.Name AS Artist
    FROM albums
    JOIN artists a on albums.ArtistId = a.ArtistId
    ORDER BY Artist;
"""

albums = pd.read_sql_query(requete, conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


Le code ci-dessous permet de créer une vue correspondant à la requête ci-dessous :    
Référence vue sous SQLite : https://www.sqlite.org/lang_createview.html

In [9]:
# creation vue albums avec artistes
creation_vue_albums = """
    CREATE VIEW v_albums (
        AlbumId,
        Title,
        Artist)
    AS
        SELECT AlbumId,
               Title,
               a.Name AS Artist
        FROM albums
        JOIN artists a on albums.ArtistId = a.ArtistId
        ORDER BY Artist, Title;
"""

executer_requete(creation_vue_albums, conn)

Une fois la vue créé, on peut la réutiliser dans des requêtes SQL comme une table:  

In [10]:
# Utilisation de la vue v_albums
albums = pd.read_sql_query("SELECT * FROM v_albums LIMIT 10;", conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


In [11]:
# Utilisation de la vue v_albums avec une clause WHERE
albums = pd.read_sql_query("SELECT * FROM v_albums WHERE Artist = 'U2';", conn)
albums

,AlbumId,Title,Artist
0,232,Achtung Baby,U2
1,233,All That You Can't Leave Behind,U2
2,234,B-Sides 1980-1990,U2
3,235,How To Dismantle An Atomic Bomb,U2
4,255,Instant Karma: The Amnesty International Campa...,U2
5,236,Pop,U2
6,237,Rattle And Hum,U2
7,238,The Best Of 1980-1990,U2
8,239,War,U2
9,240,Zooropa,U2


In [12]:
# Utilisation de la vue v_albums avec une autre jointure
requete = """
    SELECT a.AlbumId,
           a.Title,
           a.Artist,
           count(t.TrackId) as Tracks
    FROM v_albums a
    JOIN tracks t ON a.AlbumId = t.AlbumId
    GROUP BY 1, 2, 3
    ORDER BY 3, 2;
"""
albums = pd.read_sql_query(requete, conn)
albums.head()

,AlbumId,Title,Artist,Tracks
0,1,For Those About To Rock We Salute You,AC/DC,10
1,4,Let There Be Rock,AC/DC,8
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,1
3,267,Worlds,Aaron Goldberg,1
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...,2


In [13]:
# Effacer une vue
executer_requete("DROP VIEW IF EXISTS v_albums;", conn)

### Question: Avantages & Inconvénients des vues SQL

Quelles sont les avantages et inconvénients des vues SQL ?  
*A compléter*

---

## Exercices de création / mise en oeuvre de vues

### Vue 1: caractéristiques complètes des Albums  
A partir de l'exemple de vue ci-dessus, créer et exploiter une nouvelle vue v_albums reprenant pour chaque album:  
- AlbumId
- Title
- Artist
- nb de pistes
- Durée totale de l'album (en minutes)
- Prix de l'album

In [ ]:
creation_vue_albums = """
    CREATE VIEW v_albums_plus (
        AlbumId,
        Title,
        Artist,
        TrackNb,
        Minutes,
        Price)
    AS
        SELECT albums.AlbumId,
               albums.Title,
               artists.Name AS "Artist",
               COUNT(tracks.TrackId) AS "TrackNb",
               SUM(tracks.Milliseconds)/60000 AS "Minutes",
               SUM(tracks.UnitPrice) AS "Price"
        FROM albums
        JOIN artists ON albums.ArtistId = artists.ArtistId
        JOIN tracks ON albums.AlbumId = tracks.AlbumId
        GROUP BY albums.AlbumId
        ORDER BY Artist, Title;
"""

executer_requete(creation_vue_albums, conn)

In [21]:
v_albums = pd.read_sql_query("SELECT * FROM v_albums_plus;", conn)
v_albums.head(5)

,AlbumId,Title,Artist,TrackNb,Minutes,Price
0,1,For Those About To Rock We Salute You,AC/DC,10,40,9.90
1,4,Let There Be Rock,AC/DC,8,40,7.92
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,1,3,0.99
3,267,Worlds,Aaron Goldberg,1,4,0.99
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...,2,7,1.98


In [22]:
len(v_albums)

347

### Vue 2: caractéristiques détaillées des pistes
Créer et exploiter une nouvelle vue v_tracks reprenant pour chaque piste:  
- TrackId
- Name
- Genre
- Album
- Artist
- Composer
- Temps (en seconde)
- MediaType

In [43]:
executer_requete("DROP VIEW v_tracks_plus", conn) 

In [44]:
creation_vue_tracks = """
    CREATE VIEW v_tracks_plus (
        TrackId,
        Name,
        Genre,
        Album,
        Artist,
        Composer,
        Time,
        MediaType)
    AS
        SELECT tracks.TrackId ,
               tracks.Name ,
               genres.Name ,
               albums.Title ,
               artists.Name ,
               tracks.Composer ,
               (tracks.Milliseconds)/60000.,
               tracks.MediaTypeId 
        FROM tracks
        JOIN albums ON albums.AlbumId = tracks.AlbumId 
        JOIN artists ON artists.ArtistId = albums.artistId
        JOIN genres ON genres.GenreId = tracks.GenreId 
        GROUP BY tracks.TrackId;
"""

executer_requete(creation_vue_tracks, conn)

In [45]:
v_tracks = pd.read_sql_query("SELECT * FROM v_tracks_plus;", conn)
v_tracks.head(5)

,TrackId,Name,Genre,Album,Artist,Composer,Time,MediaType
0,1,For Those About To Rock (We Salute You),Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",5.728650,1
1,2,Balls to the Wall,Rock,Balls to the Wall,Accept,None,5.709367,2
2,3,Fast As a Shark,Rock,Restless and Wild,Accept,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",3.843650,2
3,4,Restless and Wild,Rock,Restless and Wild,Accept,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",4.200850,2
4,5,Princess of the Dawn,Rock,Restless and Wild,Accept,Deaffy & R.A. Smith-Diesel,6.256967,2


### Vue 3: Vue augmentée invoices
A. Créer une nouvelle vue v_invoices à partir de la table invoices ajoutant 3 nouveaux attributs basés sur l'attribut `InvoiceDate`:  
- semaine de la transaction
- mois de la transaction
- année de la transaction

In [56]:
executer_requete("DROP VIEW v_invoices", conn) 

In [57]:
creation_vue_invoices ="""
    CREATE VIEW v_invoices (
        Semaine,
        Mois,
        Année)
        
    AS
      
    SELECT STRFTIME("%w",InvoiceDate), 
            STRFTIME("%m", InvoiceDate),
            STRFTIME ("%Y", InvoiceDate)
    FROM invoices i 

"""
executer_requete(creation_vue_invoices, conn)

In [58]:
v_invoices = pd.read_sql_query("SELECT * FROM v_invoices;", conn)
v_invoices.head(5)

,Semaine,Mois,Année
0,4,01,2009
1,5,01,2009
2,6,01,2009
3,2,01,2009
4,0,01,2009


B. Exploiter la vue v_invoices ainsi crée pour déterminer les ventes annuelles par pays.

In [67]:
executer_requete("DROP VIEW v_invoices_plus", conn) 

In [68]:
creation_vue_invoices_2 ="""
    CREATE VIEW v_invoices_plus (
        Semaine,
        Mois,
        Année,
        Total,
        Pays,
        Ville,
        Departement,
        AdressePostale,
        Adresse,
        Utilisateur)
        
    AS
      
    SELECT STRFTIME("%w",InvoiceDate) as "Week", 
            STRFTIME("%m", InvoiceDate)as "Month",
            STRFTIME ("%Y", InvoiceDate)as "Year",
            SUM(Total),
            BillingCountry, 
            BillingCity,
            BillingState,
            BillingPostalCode, 
            BillingAddress,
            CustomerId 
    FROM invoices i 
    GROUP BY BillingCountry, Year
    ORDER BY BillingCountry, Year  
"""
executer_requete(creation_vue_invoices_2, conn)

In [69]:
v_invoices2 = pd.read_sql_query("SELECT * FROM v_invoices_plus", conn)
v_invoices2.head(50)

,Semaine,Mois,Année,Total,Pays,Ville,Departement,AdressePostale,Adresse,Utilisateur
0,6,06,2010,11.88,Argentina,Buenos Aires,None,1106,307 Macacha Güemes,56
1,0,08,2011,0.99,Argentina,Buenos Aires,None,1106,307 Macacha Güemes,56
2,1,01,2013,24.75,Argentina,Buenos Aires,None,1106,307 Macacha Güemes,56
3,6,04,2009,11.88,Australia,Sidney,NSW,2010,421 Bourke Street,55
4,0,05,2010,0.99,Australia,Sidney,NSW,2010,421 Bourke Street,55
5,1,11,2011,1.98,Australia,Sidney,NSW,2010,421 Bourke Street,55
6,0,01,2012,22.77,Australia,Sidney,NSW,2010,421 Bourke Street,55
7,2,12,2009,1.98,Austria,Vienne,None,1010,"Rotenturmstraße 4, 1010 Innere Stadt",7
8,1,01,2010,27.77,Austria,Vienne,None,1010,"Rotenturmstraße 4, 1010 Innere Stadt",7
9,2,04,2012,11.88,Austria,Vienne,None,1010,"Rotenturmstraße 4, 1010 Innere Stadt",7


### Vue 4: A vous de jouer
Créer et exploiter une nouvelle vue de votre choix  